In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [3]:
# Read the dataset
dataset = pd.read_json("data_without_hashtags.json")

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Read the dataset
#dataset = pd.read_json("data_without_hashtags.json")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['isSarcastic'], test_size=0.2, random_state=42)

# Tokenize and vectorize the training text data using Tokenizer and pad_sequences
max_length = 140
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

y_train_categorical = to_categorical(y_train, num_classes=2)

# Display the shapes of the resulting matrices
print("Shape of X_train:", X_train.shape)

# Tokenize and vectorize the testing text data using the same Tokenizer
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length)

# Display the shape of X_test
print("Shape of X_test:", X_test.shape)

Shape of X_train: (31824, 140)
Shape of X_test: (7956, 140)


In [6]:
from keras.models import Sequential
from keras.layers import Embedding, CuDNNLSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.optimizers import Adam

embedding_dim = 100

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 140

optimizer = Adam(learning_rate=0.000009)
m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(CuDNNLSTM(units=150))
m1.add(Dense(units=64))
m1.add(Dense(units=64))
m1.add(Dense(units=1, activation='sigmoid'))

m1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 140, 100)          3221100   
                                                                 
 cu_dnnlstm (CuDNNLSTM)      (None, 150)               151200    
                                                                 
 dense (Dense)               (None, 64)                9664      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,386,189
Trainable params: 3,386,189
Non-trainable params: 0
_________________________________________________________________


In [9]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import uniform
from keras.wrappers.scikit_learn import KerasClassifier

# Define your Keras model as a function
def create_model(learning_rate=0.001, embedding_dim=100, units=150):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(CuDNNLSTM(units=units))
    model.add(Dense(units=64))
    model.add(Dense(units=64))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model using KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=64, verbose=0)


parameters = {
    'learning_rate': uniform(0.00001, 0.00009),  # Corrected range
    'embedding_dim': [10, 15, 20, 50, 100],
    'units': [50, 100, 150, 200]
}

# Create Random Search
random_search = RandomizedSearchCV(estimator=model, param_distributions=parameters, n_iter=10, cv=StratifiedKFold(), scoring='accuracy', verbose=1)

# Perform Random Search
random_result = random_search.fit(X_train, y_train)

# Print results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))


C:\Users\Sarah\AppData\Local\Temp\ipykernel_25336\2700761065.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=64, verbose=0)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
199/199 [==============================] - 1s 4ms/step
Best: 0.753991 using {'embedding_dim': 100, 'learning_rate': 1.927457211768945e-05, 'units': 50}


In [ ]:
# Train the model
m1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')